## Introduktion till regression - Lektion 1

#### Sätta det i perspektiv

✅ Det finns många typer av regressionsmetoder, och vilken du väljer beror på vilken typ av svar du söker. Om du vill förutsäga den sannolika längden för en person i en viss ålder, skulle du använda `linjär regression`, eftersom du söker ett **numeriskt värde**. Om du är intresserad av att avgöra om en viss typ av mat ska betraktas som vegansk eller inte, söker du en **kategoriindelning** och skulle använda `logistisk regression`. Du kommer att lära dig mer om logistisk regression senare. Fundera lite på några frågor du kan ställa till data, och vilken av dessa metoder som skulle vara mest lämplig.

I det här avsnittet kommer du att arbeta med en [liten dataset om diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Föreställ dig att du ville testa en behandling för diabetiker. Maskininlärningsmodeller kan hjälpa dig att avgöra vilka patienter som skulle svara bättre på behandlingen, baserat på kombinationer av variabler. Även en mycket enkel regressionsmodell, när den visualiseras, kan visa information om variabler som kan hjälpa dig att organisera dina teoretiska kliniska studier.

Med det sagt, låt oss sätta igång med denna uppgift!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Konstverk av @allison_horst</figcaption>

<!--![Konstverk av \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Konstverk av @allison_horst-->


## 1. Ladda upp vårt verktygspaket

För den här uppgiften behöver vi följande paket:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) är en [samling av R-paket](https://www.tidyverse.org/packages) som är utformade för att göra dataanalys snabbare, enklare och roligare!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) är ett ramverk som består av en [samling paket](https://www.tidymodels.org/packages/) för modellering och maskininlärning.

Du kan installera dem med följande kommando:

`install.packages(c("tidyverse", "tidymodels"))`

Skriptet nedan kontrollerar om du har de paket som krävs för att slutföra denna modul och installerar dem åt dig om några saknas.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Nu, låt oss ladda dessa fantastiska paket och göra dem tillgängliga i vår nuvarande R-session. (Detta är bara för illustration, `pacman::p_load()` har redan gjort det åt dig)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabetes-datasetet

I den här övningen ska vi använda våra regressionskunskaper för att göra förutsägelser på ett diabetes-dataset. [Diabetes-datasetet](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) innehåller `442 prover` med data relaterad till diabetes, med 10 prediktorvariabler: `ålder`, `kön`, `kroppsmasseindex`, `genomsnittligt blodtryck` och `sex blodserummätningar` samt en utfallsvariabel `y`: ett kvantitativt mått på sjukdomsprogression ett år efter baslinjen.

|Antal observationer|442|
|-------------------|:---|
|Antal prediktorer|De första 10 kolumnerna är numeriska prediktorer|
|Utfall/Mål|Kolumn 11 är ett kvantitativt mått på sjukdomsprogression ett år efter baslinjen|
|Information om prediktorer|- ålder i år
||- kön
||- bmi kroppsmasseindex
||- bp genomsnittligt blodtryck
||- s1 tc, totalt serumkolesterol
||- s2 ldl, lågdensitetslipoproteiner
||- s3 hdl, högdensitetslipoproteiner
||- s4 tch, totalt kolesterol / HDL
||- s5 ltg, möjligen logaritmen av serumtriglyceridnivå
||- s6 glu, blodsockernivå|

> 🎓 Kom ihåg, detta är övervakad inlärning, och vi behöver ett namngivet mål 'y'.

Innan du kan manipulera data med R, måste du importera data till R:s minne eller skapa en anslutning till data som R kan använda för att komma åt den på distans.

> Paketet [readr](https://readr.tidyverse.org/), som är en del av Tidyverse, erbjuder ett snabbt och användarvänligt sätt att läsa in rektangulära data i R.

Nu ska vi ladda diabetes-datasetet från denna käll-URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Vi ska också göra en snabb kontroll av vår data med hjälp av `glimpse()` och visa de första 5 raderna med `slice()`.

Innan vi går vidare, låt oss introducera något du ofta kommer att stöta på i R-kod 🥁🥁: pipe-operatorn `%>%`

Pipe-operatorn (`%>%`) utför operationer i logisk sekvens genom att skicka ett objekt vidare till en funktion eller ett uttryck. Du kan tänka på pipe-operatorn som att säga "och sedan" i din kod.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` visar oss att denna data har 442 rader och 11 kolumner, där alla kolumner är av datatypen `double`.

<br>

> glimpse() och slice() är funktioner i [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, som är en del av Tidyverse, är en grammatik för datamanipulation som erbjuder en konsekvent uppsättning verb för att lösa de vanligaste utmaningarna inom datamanipulation.

<br>

Nu när vi har datan, låt oss fokusera på en specifik variabel (`bmi`) som mål för denna övning. Detta kräver att vi väljer ut de önskade kolumnerna. Så, hur gör vi detta?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) låter oss *välja* (och eventuellt byta namn på) kolumner i en data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Tränings- och testdata

Det är vanligt inom övervakad inlärning att *dela upp* data i två delmängder; en (vanligtvis större) uppsättning för att träna modellen, och en mindre "håll-ut" uppsättning för att se hur modellen presterade.

Nu när vi har data redo kan vi se om en maskin kan hjälpa till att avgöra en logisk uppdelning mellan siffrorna i detta dataset. Vi kan använda paketet [rsample](https://tidymodels.github.io/rsample/), som är en del av Tidymodels-ramverket, för att skapa ett objekt som innehåller information om *hur* man delar upp data, och sedan två ytterligare rsample-funktioner för att extrahera de skapade tränings- och testuppsättningarna:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Träna en linjär regressionsmodell med Tidymodels

Nu är vi redo att träna vår modell!

I Tidymodels specificerar du modeller med `parsnip()` genom att ange tre koncept:

-   Modellens **typ** skiljer mellan olika modeller som linjär regression, logistisk regression, beslutsträd och så vidare.

-   Modellens **läge** inkluderar vanliga alternativ som regression och klassificering; vissa modelltyper stödjer båda dessa medan andra bara har ett läge.

-   Modellens **motor** är det beräkningsverktyg som kommer att användas för att anpassa modellen. Ofta är dessa R-paket, såsom **`"lm"`** eller **`"ranger"`**

Denna modellinformation fångas i en modelspecifikation, så låt oss skapa en!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Efter att en modell har *specificerats* kan modellen `estimeras` eller `tränas` med hjälp av funktionen [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), vanligtvis med en formel och lite data.

`y ~ .` betyder att vi kommer att anpassa `y` som den förutsagda kvantiteten/målet, förklarad av alla prediktorer/funktioner, dvs. `.` (i det här fallet har vi bara en prediktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Från modellens output kan vi se de koefficienter som lärdes in under träningen. Dessa representerar koefficienterna för den bästa anpassade linjen som ger oss det lägsta totala felet mellan den faktiska och den förutsagda variabeln.
<br>

## 5. Gör förutsägelser på testuppsättningen

Nu när vi har tränat en modell kan vi använda den för att förutsäga sjukdomsprogressionen y för testdatamängden med hjälp av [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Detta kommer att användas för att dra linjen mellan datagrupper.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Vi har precis tränat en modell och använt den för att göra förutsägelser!

När man gör förutsägelser är tidymodels-konventionen att alltid skapa en tibble/data frame med resultat och standardiserade kolumnnamn. Detta gör det enkelt att kombinera den ursprungliga datan med förutsägelserna i ett användbart format för efterföljande operationer, såsom att skapa diagram.

`dplyr::bind_cols()` binder effektivt flera data frames kolumnvis.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Visa modellresultat

Nu är det dags att se detta visuellt 📈. Vi ska skapa ett spridningsdiagram med alla `y`- och `bmi`-värden från testuppsättningen och sedan använda förutsägelserna för att rita en linje på den mest lämpliga platsen, mellan modellens datagrupperingar.

R har flera system för att skapa grafer, men `ggplot2` är ett av de mest eleganta och mångsidiga. Det gör det möjligt att komponera grafer genom att **kombinera oberoende komponenter**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Fundera lite på vad som händer här. En rak linje går genom många små datapunkter, men vad gör den egentligen? Kan du se hur du borde kunna använda denna linje för att förutsäga var en ny, osedd datapunkt borde passa i förhållande till diagrammets y-axel? Försök att formulera den praktiska användningen av denna modell.

Grattis, du har byggt din första linjära regressionsmodell, gjort en förutsägelse med den och visat den i ett diagram!



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på sitt originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
